<a href="https://colab.research.google.com/github/Ricardobigdata/PrimerRepor/blob/main/C%C3%B3pia_de_auditoria_cap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
from pandas import DataFrame
from datetime import datetime
import pandas as pd
import numpy as np
import time
import warnings


In [ ]:
pasta_raiz = '/content/drive/MyDrive/AUDITORIA/2023'

In [466]:
movimentos = pd.read_csv(f'{pasta_raiz}/ValoresMovimentoTitulos.csv', sep=';', encoding='utf-8')
movimentos["valor"] = [x.replace(",", ".") for x in movimentos["vdrValor"]]
movimentos["movimento"] = movimentos["valor"].astype(float, errors = 'raise')
movimentos["mrfMesAno2"] = pd.to_datetime(movimentos['mrfMesAno'])

In [467]:
provisao = pd.read_csv(f'{pasta_raiz}/ValoresProvTit.csv', sep=';', encoding='utf-8')
provisao["valor"] = [x.replace(".", "") for x in provisao[' vdrValor ']]
provisao["valor1"] = [x.replace(",", ".") for x in provisao["valor"]]
provisao["provisao"] = provisao["valor1"].astype(float, errors = 'raise')
provisao["mrfMesAno2"] = pd.to_datetime(provisao['mrfMesAno'])


In [468]:
resumo_mov = movimentos.groupby(['plnCodigo','mrfMesAno2','cmpID'])['movimento'].sum().apply(lambda x:"{:,.2f}".format(x))
resumo_prov = provisao.groupby(['plnCodigo','mrfMesAno2','cmpID'])['provisao'].sum().apply(lambda x:"{:,.2f}".format(x))
resumo_mov = pd.DataFrame(resumo_mov).reset_index()
resumo_prov = pd.DataFrame(resumo_prov).reset_index()

In [469]:
empilhado_prov_tot = resumo_prov.query('mrfMesAno2 >= "2021-01-12" ' )
empilhado_mov_tot = resumo_mov.query('mrfMesAno2 >= "2021-01-12" ' )

## Geração dos valores de provisões

In [470]:
PMC = empilhado_prov_tot.query('cmpID == 12781')
PDB = empilhado_prov_tot.query('cmpID == 12782')
PRA = empilhado_prov_tot.query('cmpID == 12785')
PRV = empilhado_prov_tot.query('cmpID == 12784')
PSR = empilhado_prov_tot.query('cmpID == 12786')
PCS = empilhado_prov_tot.query('cmpID == 12787')
PSRC = empilhado_prov_tot.query('cmpID == 12791')
PSP = empilhado_prov_tot.query('cmpID == 12793')
PDA = empilhado_prov_tot.query('cmpID == 12794')
OPT = empilhado_prov_tot.query('cmpID == 12795')

OPT = OPT.rename(columns={'provisao': 'OPT'})
PMC = PMC.rename(columns={'provisao': 'PMC'})
PRA = PRA.rename(columns={'provisao': 'PRA'})
PDB = PDB.rename(columns={'provisao': 'PDB'})
PRV = PRV.rename(columns={'provisao': 'PRV'})
PSR = PSR.rename(columns={'provisao': 'PSR'})
PCS = PCS.rename(columns={'provisao': 'PCS'})
PSRC = PSRC.rename(columns={'provisao': 'PSRC'})
PSP = PSP.rename(columns={'provisao': 'PSP'})
PDA = PDA.rename(columns={'provisao': 'PDA'})

## Geração dos valores de movimentos das provisões

In [471]:
ARREC_INST = empilhado_mov_tot.query('cmpID == 12788')
ARREC_INST = ARREC_INST.rename(columns={'movimento': 'ARREC_INST'})
SORTEIO_REALIZADOS_INST = empilhado_mov_tot.query('cmpID == 12789')
SORTEIO_REALIZADOS_INST = SORTEIO_REALIZADOS_INST.rename(columns={'movimento': 'SORTEIO_REALIZADOS_INST'})
SORTEIO_A_REALIZAR_C_OBRIG = empilhado_mov_tot.query('cmpID == 12790')
SORTEIO_A_REALIZAR_C_OBRIG = SORTEIO_A_REALIZAR_C_OBRIG.rename(columns={'movimento': 'SORTEIO_A_REALIZAR_C_OBRIG'})
ARREC_N_CAP = empilhado_mov_tot.query('cmpID == 12808')
ARREC_N_CAP = ARREC_N_CAP.rename(columns={'movimento': 'ARREC_N_CAP'})
ARREC_CAP = empilhado_mov_tot.query('cmpID == 12809')
ARREC_CAP = ARREC_CAP.rename(columns={'movimento': 'ARREC_CAP'})
TRANSF_PMC_P_PR = empilhado_mov_tot.query('cmpID == 12810')
TRANSF_PMC_P_PR = TRANSF_PMC_P_PR.rename(columns={'movimento': 'TRANSF_PMC_P_PR'})
AJUST_PMC = empilhado_mov_tot.query('cmpID == 12811')
AJUST_PMC = AJUST_PMC.rename(columns={'movimento': 'AJUST_PMC'})
PENALT = empilhado_mov_tot.query('cmpID == 12812')
PENALT = PENALT.rename(columns={'movimento': 'PENALT'})
ARREC_BONUS = empilhado_mov_tot.query('cmpID == 12813')
ARREC_BONUS = ARREC_BONUS.rename(columns={'movimento': 'ARREC_BONUS'})
TRANSF_PDB_P_PR = empilhado_mov_tot.query('cmpID == 12814')
TRANSF_PDB_P_PR = TRANSF_PDB_P_PR.rename(columns={'movimento': 'TRANSF_PDB_P_PR'})
AJUST_PDB = empilhado_mov_tot.query('cmpID == 12815')
AJUST_PDB = AJUST_PDB.rename(columns={'movimento': 'AJUST_PDB'})
RESGATADO = empilhado_mov_tot.query('cmpID == 12816')
RESGATADO = RESGATADO.rename(columns={'movimento': 'RESGATADO'})
PRESCRICAO = empilhado_mov_tot.query('cmpID == 12817')
PRESCRICAO = PRESCRICAO.rename(columns={'movimento': 'PRESCRICAO'})
AJUST_PR = empilhado_mov_tot.query('cmpID == 12818')
AJUST_PR = AJUST_PR.rename(columns={'movimento': 'AJUST_PR'})
DEV_CUST_SORT_FUTURO = empilhado_mov_tot.query('cmpID == 12819')
DEV_CUST_SORT_FUTURO = DEV_CUST_SORT_FUTURO.rename(columns={'movimento': 'DEV_CUST_SORT_FUTURO'})
ARREC_SORT = empilhado_mov_tot.query('cmpID == 12820')
ARREC_SORT = ARREC_SORT.rename(columns={'movimento': 'ARREC_SORT'})
BAIXA_PSR = empilhado_mov_tot.query('cmpID == 12821')
BAIXA_PSR = BAIXA_PSR.rename(columns={'movimento': 'BAIXA_PSR'})
SORT_REALIZADO = empilhado_mov_tot.query('cmpID == 12822')
SORT_REALIZADO = SORT_REALIZADO.rename(columns={'movimento': 'SORT_REALIZADO'})
SORT_PAGOS = empilhado_mov_tot.query('cmpID == 12823')
SORT_PAGOS = SORT_PAGOS.rename(columns={'movimento': 'SORT_PAGOS'})
AJUST_PSP_PRESCRICAO = empilhado_mov_tot.query('cmpID == 12824')
AJUST_PSP_PRESCRICAO = AJUST_PSP_PRESCRICAO.rename(columns={'movimento': 'AJUST_PSP_PRESCRICAO'})
AJUST_PSP_OUTROS = empilhado_mov_tot.query('cmpID == 12825')
AJUST_PSP_OUTROS = AJUST_PSP_OUTROS.rename(columns={'movimento': 'AJUST_PSP_OUTROS'})

In [472]:
# Construção da validação da PMC

ARREC = PMC.merge(ARREC_CAP[['ARREC_CAP','plnCodigo','mrfMesAno2']], how='left', on=["plnCodigo","mrfMesAno2"])
TRANS = ARREC.merge(TRANSF_PMC_P_PR[['TRANSF_PMC_P_PR','plnCodigo','mrfMesAno2']], how='left', on=["plnCodigo","mrfMesAno2"])
ajust = TRANS.merge(AJUST_PMC[['AJUST_PMC','plnCodigo','mrfMesAno2']], how='left', on=["plnCodigo","mrfMesAno2"])
MOV_PMC = ajust.merge(PENALT[['PENALT','plnCodigo','mrfMesAno2']], how='left', on=["plnCodigo","mrfMesAno2"])

In [473]:
MOV_PMC['mrfMesAno_1'] = MOV_PMC["PMC"].shift(1)
MOV_PMC.loc[MOV_PMC['mrfMesAno2'] == '2021-01-12', 'PMC_1'] = 0  
MOV_PMC.loc[MOV_PMC['mrfMesAno2'] != '2021-01-12', 'PMC_1'] = MOV_PMC['mrfMesAno_1']

In [474]:
juros = 0.005
tr={'mrfMesAno': 
    ['2021-01-12','2022-01-01','2022-01-02','2022-01-03','2022-01-04','2022-01-05','2022-01-06','2022-01-07','2022-01-08','2022-01-09','2022-01-10','2022-01-11','2022-01-12'], 
'tr': [0,0.000605,0,0.000971,0.000555,0.001663,0.001484,0.001631,0.002409,0.00180,0.001494,0.001507,0.002072]
}
tr_ = pd.DataFrame(data=tr)
tr_["mrfMesAno2"] = pd.to_datetime(tr_['mrfMesAno'])


In [475]:
MOV_PMC = MOV_PMC.merge(tr_, how='left', on="mrfMesAno2")

In [502]:
MOV_PMC['ARREC_CAP'] = pd.to_numeric(MOV_PMC['ARREC_CAP'], downcast='float', errors='coerce')
MOV_PMC['TRANSF_PMC_P_PR'] = pd.to_numeric(MOV_PMC['TRANSF_PMC_P_PR'], downcast='float', errors='coerce')
MOV_PMC['AJUST_PMC'] = pd.to_numeric(MOV_PMC['AJUST_PMC'], downcast='float', errors='coerce')
MOV_PMC['PENALT'] = pd.to_numeric(MOV_PMC['PENALT'], downcast='float', errors='coerce')
MOV_PMC['PMC_1'] = pd.to_numeric(MOV_PMC['PMC_1'], downcast='float', errors='coerce')
MOV_PMC['PMC'] = pd.to_numeric(MOV_PMC['PMC'], downcast='float', errors='coerce')



fator1 = (1+float(f'{juros}'))*(1+MOV_PMC['tr'])
fator2 = ((1+float(f'{juros}'))**0.5)*(1+MOV_PMC['tr'])**0.5

MOV_PMC['calculo']  = (MOV_PMC['PMC_1']*fator1) + ((MOV_PMC['ARREC_CAP'] - MOV_PMC['TRANSF_PMC_P_PR']  - MOV_PMC['AJUST_PMC'] - MOV_PMC['PENALT'])*fator2)

MOV_PMC['controle'] = (MOV_PMC.PMC+MOV_PMC.ARREC_CAP+MOV_PMC.TRANSF_PMC_P_PR+MOV_PMC.AJUST_PMC+MOV_PMC.PENALT+MOV_PMC.PMC_1)

MOV_PMC.query('mrfMesAno2 !=  "2021-01-12" & controle > 0 ')


,plnCodigo,mrfMesAno2,cmpID,PMC,ARREC_CAP,TRANSF_PMC_P_PR,AJUST_PMC,PENALT,mrfMesAno_1,PMC_1,mrfMesAno,tr,calculo,controle
460,65,2022-01-01,12781,312.309998,157.080002,148.029999,0.0,0.0,302.59,302.589996,2022-01-01,0.000605,313.362272,920.010010
461,65,2022-01-02,12781,320.119995,162.800003,155.470001,0.0,0.0,312.31,312.309998,2022-01-02,0.000000,321.219852,950.700012
462,65,2022-01-03,12781,315.019989,151.800003,157.729996,0.0,0.0,320.12,320.119995,2022-01-03,0.000971,316.085301,944.669983
463,65,2022-01-04,12781,305.250000,153.119995,163.570007,0.0,0.0,315.02,315.019989,2022-01-04,0.000555,306.291788,936.959961
464,65,2022-01-05,12781,325.220001,171.600006,152.630005,0.0,0.0,305.25,305.250000,2022-01-05,0.001663,326.319592,954.700012
465,65,2022-01-06,12781,344.170013,172.039993,154.089996,0.0,0.0,325.22,325.220001,2022-01-06,0.001484,345.339304,995.520020
466,65,2022-01-07,12781,313.839996,141.240005,172.660004,0.0,0.0,344.17,344.170013,2022-01-07,0.001631,314.930885,971.910034
2026,239,2022-01-01,12781,406.809998,211.449997,178.139999,0.0,0.0,372.68,372.679993,2022-01-01,0.000605,408.173260,1169.080078
2027,239,2022-01-02,12781,429.609985,217.820007,195.669998,0.0,0.0,406.81,406.809998,2022-01-02,0.000000,431.049363,1249.909912
2028,239,2022-01-03,12781,437.019989,218.639999,212.339996,0.0,0.0,429.61,429.609985,2022-01-03,0.000971,438.496071,1297.609985


In [499]:
MOV_PMC.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12091 entries, 0 to 12090
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   plnCodigo        12091 non-null  int64         
 1   mrfMesAno2       12091 non-null  datetime64[ns]
 2   cmpID            12091 non-null  int64         
 3   PMC              12091 non-null  object        
 4   ARREC_CAP        3555 non-null   float32       
 5   TRANSF_PMC_P_PR  3564 non-null   float32       
 6   AJUST_PMC        4372 non-null   float32       
 7   PENALT           4372 non-null   float32       
 8   mrfMesAno_1      12090 non-null  object        
 9   PMC_1            10645 non-null  float32       
 10  mrfMesAno        12091 non-null  object        
 11  tr               12091 non-null  float64       
 12  calculo          2797 non-null   float64       
dtypes: datetime64[ns](1), float32(5), float64(2), int64(2), object(3)
memory usage: 1.1+ MB


In [416]:
MOV_PMC

,plnCodigo,mrfMesAno2,cmpID,PMC,ARREC_CAP,TRANSF_PMC_P_PR,AJUST_PMC,PENALT,mrfMesAno_1,PMC_1,calculo,mrfMesAno3
0,2,2021-01-12,12781,0.00,0.0,0.0,0.0,0.0,NaN,0.0,0.0,2021-01-12
1,2,2022-01-01,12781,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,2022-01-01
2,2,2022-01-02,12781,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,2022-01-02
3,2,2022-01-03,12781,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,2022-01-03
4,2,2022-01-04,12781,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,2022-01-04
...,...,...,...,...,...,...,...,...,...,...,...,...
12086,1499,2022-01-08,12781,"336,756.00",NaN,NaN,NaN,NaN,"384,544.53",NaN,NaN,2022-01-08
12087,1500,2022-01-08,12781,"161,693.49",NaN,NaN,NaN,NaN,"336,756.00",NaN,NaN,2022-01-08
12088,1501,2022-01-08,12781,"234,000.00",NaN,NaN,NaN,NaN,"161,693.49",NaN,NaN,2022-01-08
12089,1502,2022-01-08,12781,"132,240.00",NaN,NaN,NaN,NaN,"234,000.00",NaN,NaN,2022-01-08


In [ ]:

pd.to_numeric(MOV_PMC['ARREC_CAP'], errors='ignore')

0        0.00
1        0.00
2        0.00
3        0.00
4        0.00
         ... 
12086     NaN
12087     NaN
12088     NaN
12089     NaN
12090     NaN
Name: ARREC_CAP, Length: 12091, dtype: object

In [ ]:
# Construção da validação da PR

PR_TRAN = PRA.merge(TRANSF_PMC_P_PR[['TRANSF_PMC_P_PR','plnCodigo','mrfMesAno2']], how='left', on=["plnCodigo","mrfMesAno2"])
TRANSF_ = PR_TRAN.merge(TRANSF_PDB_P_PR[['TRANSF_PDB_P_PR','plnCodigo','mrfMesAno2']], how='left', on=["plnCodigo","mrfMesAno2"])
DEV_CUSTO = TRANSF_.merge(DEV_CUST_SORT_FUTURO[['DEV_CUST_SORT_FUTURO','plnCodigo','mrfMesAno2']], how='left', on=["plnCodigo","mrfMesAno2"])
RESGATADO = DEV_CUSTO.merge(RESGATADO[['RESGATADO','plnCodigo','mrfMesAno2']], how='left', on=["plnCodigo","mrfMesAno2"])
PRESCRICAO = RESGATADO.merge(PRESCRICAO[['PRESCRICAO','plnCodigo','mrfMesAno2']], how='left', on=["plnCodigo","mrfMesAno2"])
MOV_PR = PRESCRICAO.merge(AJUST_PR[['AJUST_PR','plnCodigo','mrfMesAno2']], how='left', on=["plnCodigo","mrfMesAno2"])


In [ ]:
# Construção da validação da PSR

ARREC_PSR = PSR.merge(ARREC_SORT[['ARREC_SORT','plnCodigo','mrfMesAno2']], how='left', on=["plnCodigo","mrfMesAno2"])
MOV_PSR = ARREC_PSR.merge(BAIXA_PSR[['BAIXA_PSR','plnCodigo','mrfMesAno2']], how='left', on=["plnCodigo","mrfMesAno2"])


In [ ]:
# Construção da validação da PSP

PSP_SORT= PSR.merge(SORT_REALIZADO[['SORT_REALIZADO','plnCodigo','mrfMesAno2']], how='left', on=["plnCodigo","mrfMesAno2"])
MOV = PSP_SORT.merge(SORT_PAGOS[['SORT_PAGOS','plnCodigo','mrfMesAno2']], how='left', on=["plnCodigo","mrfMesAno2"])
MOV2 = MOV.merge(AJUST_PSP_PRESCRICAO[['AJUST_PSP_PRESCRICAO','plnCodigo','mrfMesAno2']], how='left', on=["plnCodigo","mrfMesAno2"])
MOV_PSP = MOV2.merge(AJUST_PSP_OUTROS[['AJUST_PSP_OUTROS','plnCodigo','mrfMesAno2']], how='left', on=["plnCodigo","mrfMesAno2"])


In [ ]:
MOV_PMC.to_csv("/content/drive/MyDrive/AUDITORIA/2023/MOV_PMC.csv")

,plnCodigo,mrfMesAno2,cmpID,PMC,ARREC_CAP,TRANSF_PMC_P_PR,AJUST_PMC,PENALT,mrfMesAno_1
441,63,2021-01-12,12781,"6,682.69","3,341.10","3,040.78",0.00,0.00,0
442,63,2022-01-01,12781,"6,671.70","3,321.30","3,347.03",0.00,0.00,"6,682.69"
443,63,2022-01-02,12781,"6,537.45","3,208.90","3,353.83",0.00,0.00,"6,671.70"
444,63,2022-01-03,12781,"6,294.51","3,075.40","3,335.18",0.00,0.00,"6,537.45"
445,63,2022-01-04,12781,"6,294.47","3,210.50","3,224.11",0.00,0.00,"6,294.51"
446,63,2022-01-05,12781,"6,400.33","3,177.40","3,092.09",0.00,0.00,"6,294.47"
447,63,2022-01-06,12781,"6,378.36","3,189.20","3,230.92",0.00,0.00,"6,400.33"
448,63,2022-01-07,12781,"6,311.98","3,110.50","3,197.03",0.00,0.00,"6,378.36"
449,63,2022-01-08,12781,"6,261.48",NaN,NaN,NaN,NaN,"6,311.98"
450,64,2021-01-12,12781,"5,402.66","2,488.81","2,853.21",0.00,0.00,0
